In [ ]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

# Test the type of distribution of data for any distribution
from scipy.stats import kstest
# Test for the Heteroscedasticity
from scipy.stats import levene
# Test with nonparametric test for comparing of two
from scipy.stats import wilcoxon
# Test with nonparametric test for comparing of three or more
from scipy.stats import friedmanchisquare
# probability plot
from scipy.stats import probplot

np.set_printoptions(linewidth=100000000, formatter={'all': lambda x: str(x)})
%matplotlib notebook

In [ ]:
dims, algos, alpha = [10, 30, 50], ['BA', 'ABA', 'SABA', 'HBA', 'HSABA'], 0.05

# CEC 2017 algorithm speed test
Example of time execution speed of algorithm

In [ ]:
sys.path.append('cec2017')
from cec2017 import run_fun
from run_cec import MinMB

from NiaPy.algorithms import AlgorithmUtility
from NiaPy.task import StoppingTask, Utility

def testOne(x=0.55):
   for i in range(10 ** 6): 
      x = x + x
      x = x / 2
      x = x * x
      x = np.sqrt(x)
      x = np.log(x)
      x = np.exp(x)
      x = x / (x + 2)

def testTwo(d):
   for i in range(2 * 10 ** 5): 
      x = np.random.uniform(-100, 100, d)
      run_fun(x, 18)

def testThreeCec(a, d, fnum=1):
   autil = AlgorithmUtility()
   task = StoppingTask(D=d, nFES=d * 1e4, benchmark=MinMB(run_fun, fnum=fnum))
   algo = autil.get_algorithm(a)()
   start_time = time.time()
   best = algo.run(task)
   return time.time() - start_time, best

def testThreeBasic(a, d, fnum=1):
   mapper = {1: 'ackley'}
   task = StoppingTask(D=d, nFES=d * 1e4, benchmark=mapper[fnum])
   algo = autil.get_algorithm(a)()
   start_time = time.time()
   best = algo.run(task)
   return time.time() - start_time, best

## Test one

In [ ]:
start_time = time.time()
testOne()
t0 = (time.time() - start_time)
print(t0)

## Test two

In [ ]:
t1 = []
for d in dims:
   start_time = time.time()
   testTwo(d)
   t1.append(time.time() - start_time)
print(t1)

## Test three

In [ ]:
d = 10
# for d in dims:
for f in range(10):
   for a in algos: print('%s\t on function %d => \t%f' % (a, f+1, testThreeCec(a, d, f+1)[1][1]))
   print('--------------------------------------')

# CEC 2017 statistic
For getting the data need to perform statistic test we used: `for i in {1..30}; do python run_cec.py -c 17 -o T -rn 50 -a DE -D 10 -f $i -seed {1000..1050}; done`.
The example shows how to run DE algorithm on all benchmark functions that have problem dimensionality set to 10.
DE algorithm runs 50 times on each benchmark functions with seed in range from 1000 to 1050.
Every algorithm run has it's own seed.
We used `-o T` for generating the output.

## Example of multiple runs on one problem
* dim $\in \{10, 30, 50\}$
* fnum $\in \{1, \cdots , 30\}$
* algos $\in$ `{dynNpMsjDE, BBFWA, DE, jDE, SCA, ES(1+1), ES(m+1), ASO, BA, dynFWAG}`

## Load data
We will create a new variable `data`, that holds the optization results for 50 runs, 30 functions and specified algorithms.
Now first index presents algorithm, second index presents function and third index presents the value of optimization.

In [ ]:
dim, data = dims[0], []
data = np.asanyarray([[np.loadtxt('data/%s_%d_%d_v' % (a, fnum, dim)) for fnum in range(1, 31)] for a in algos])
# Get basic statistics
vals = []
for fnum in range(30):
   tmp = []
   print ('\nfun_num: %d' % (fnum + 1))
   for i, a in enumerate(algos):
      d = data[i, fnum] = data[i, fnum] - (fnum + 1) * 100
      print ('%10s:\tmin: %.3E \tmean: %.3E\tstd: %.3E' % (a, np.min(d), np.mean(d), np.std(d)))
      tmp.append((np.min(d), np.mean(d), np.std(d)))
   vals.append(tmp)
vals = np.asanyarray(vals)
# Get best values for basic statistics
imin, imean, istd = [], [], []
for fnum in range(30):
   imin.append(np.argmin([vals[fnum, i, 0] for i in range(len(algos))]))
   imean.append(np.argmin([vals[fnum, i, 1] for i in range(len(algos))]))
   istd.append(np.argmin([vals[fnum, i, 2] for i in range(len(algos))]))
# Generate table entrys for latex
out = ''
for i in range(len(algos)):
   for fnum in range(30): out += ('%.3E' if i != imin[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 0] + ' & ' + ('%.3E' if i != imean[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 1] + ' & ' + ('%.3E' if i != istd[fnum] else '\\textbf{%.3E}') % vals[fnum, i, 2] + ' \\\\ \n'
   out += '\n'
print ('\n', out)

## Normalize the data
Now we transform the presentation of data in our new array called `ndata` that has normalized data.
Now first index presents function, second index presents algorithm and third index presents the value of optimization.

In [ ]:
ndata = np.asanyarray([[(data[j, i] - np.min(data[j, i])) / (np.max(data[j, i]) - np.min(data[j, i])) for j in range(len(algos))] for i in range(30)])
ndata[np.isnan(ndata)] = 0

## Test for normal distribution
[Kolmogorov–Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) test: Tests whether a sample is drawn from a given distribution, or whether two samples are drawn from the same distribution.

If $\text{p-value} > \alpha$ then values belong to selected distribution.

In [ ]:
for i, a in enumerate(algos):
   tmp = ''
   for j in range(30):
      s = kstest(ndata[j, i], 'norm')
      print('%10s on %2d.: %.3E %.3E' % (a, j + 1, s[0], s[1]))
      tmp += '%.3E & ' % s[1]
   print(tmp, '\n')

### Probability Plot for one algorithm

In [ ]:
for i in range(30):
   fig = plt.figure()
   ax = fig.add_subplot(111)
   probplot(ndata[i, 0], plot=ax)
plt.show()

## Test for Heteroscedasticity
The Levene test tests the null hypothesis that all input samples are from populations with equal variances. Levene’s test is an alternative to Bartlett’s test bartlett in the case where there are significant deviations from normality.

In [ ]:
tmp = ''
for j in range(30):
   s = levene(*ndata[j, :], center='mean')
   print('%2d.: %.3E %.3E' % (j + 1, s[0], s[1]))
   tmp += '%.3E & ' % s[1]
print(tmp)

## Run Wilcoxon test
[Wilcoxon signed-rank](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) test: tests whether matched pair samples are drawn from populations with different mean ranks

In [ ]:
for j in range(30):
   tmp = ''
   print ('f%2d' % (j + 1))
   for i, a in enumerate(algos):
      s = wilcoxon(ndata[j, 0], ndata[j, i])
      print ('%10s vs. %10s on %2d. fun: %.3E %.3E' % (algos[0], a, j, s[0], s[1]))
      tmp += '%.3E & ' % s[1]
   print (tmp, '\n')

## Run Friedman test
[Friedman two-way analysis of variance by ranks](https://en.wikipedia.org/wiki/Friedman_test): tests whether k treatments in randomized block designs have identical effects

In [ ]:
for j in range(30):
   print (algos, ' on ', j + 1, ' fun: ', friedmanchisquare(*ndata[j, :]))